In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import math

In [2]:
review_distribution = pd.read_json('../full_dataset/visit_distribution_referenceid.json')
review_distribution['merge_token'] = review_distribution.apply(lambda x: x['reference_id'] + ',' + str(x['authority_stars']) + ',' + str(x['distance']), axis = 1)

In [3]:
d_before = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 0])
d_before.columns=['count_before', 'authority_stars', 'distance', 'reference_id', 'time_distribution', 'merge_token']
d_after = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 1][['merge_token', '0']])
d_after.columns = ['merge_token', 'count_after']

In [4]:
factor = pd.merge(d_before, d_after, on = 'merge_token')[['reference_id', 'authority_stars', 'distance', 'count_before', 'count_after']]
factor['factor'] = factor.apply(lambda x: x['count_after'] / x['count_before'], axis = 1)
factor.head()

,reference_id,authority_stars,distance,count_before,count_after,factor
0,-2k0to0euEaCvZeCwWhZkg,3,3,1,2,2.000000
1,-Eay214ERmVOVslWgW4xeA,5,5,3,3,1.000000
2,-t2V_fsfRSqYYsAlWCCp-Q,3,4,1,11,11.000000
3,9ag8JtM8cb8CWyhCFOIXGg,4,4,3,14,4.666667
4,9cdVdIDYg5-UDj1gHh8lsQ,2,2,9,6,0.666667


In [5]:
aggregate_result = factor[['authority_stars', 'distance', 'factor']].groupby(['authority_stars', 'distance'], as_index = False)['factor'].agg({'size': np.size, 'factor': np.mean, 'sigma': np.std })
aggregate_result.loc[aggregate_result['size'] > 3]

,authority_stars,distance,sigma,factor,size
0,1,2,1.906916,1.747192,27
1,1,3,5.337252,3.227980,38
2,1,4,6.140742,5.026908,33
3,1,5,6.117857,4.636640,18
4,2,2,3.592807,2.426426,72
5,2,3,13.816896,6.892804,68
6,2,4,8.285933,5.308815,64
7,2,5,6.657379,4.197111,26
8,3,1,0.288675,0.750000,4
9,3,2,2.723903,1.914644,200


In [6]:
factor['weight'] = factor.apply(lambda x: x['count_before'] + x['count_after'], axis = 1)
def weighted_avg_and_std(values):
    """
    Return the weighted average and biased weighted standard deviation.

    Adapted from: http://stackoverflow.com/questions/2413522/weighted-standard-deviation-in-numpy
    """
    
    average = np.average(values, weights=factor.loc[values.index, 'weight'])
    variance = np.average((values-average)**2, weights=factor.loc[values.index, 'weight'])  # Fast and numerically precise
    return (average, math.sqrt(variance))

In [7]:
agg_weighted = factor[['authority_stars', 'distance', 'factor', 'weight']].groupby(['authority_stars', 'distance'], as_index = False).agg({ 'factor': weighted_avg_and_std, 'weight': np.sum })
agg_weighted.loc[aggregate_result['size'] > 3]

,authority_stars,distance,factor,weight
0,1,2,"(1.62293456085, 1.5487131567236587)",360
1,1,3,"(4.88913000476, 5.15010078955669)",2621
2,1,4,"(8.83039600883, 7.165793396028185)",1503
3,1,5,"(10.9163509603, 10.226303214447077)",181
4,2,2,"(3.01249951927, 4.3537840696860695)",958
5,2,3,"(8.27860999799, 14.517328210860631)",5173
6,2,4,"(5.70846826608, 8.129109199134955)",2870
7,2,5,"(7.70132377039, 9.754772646231892)",354
8,3,1,"(0.7, 0.2449489742783178)",10
9,3,2,"(2.52917008076, 3.7149762679240474)",3048
